In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Airline Data Analysis") \
    .getOrCreate()

# Reload the DataFrame from HDFS (optional if you still have `df`)
df = spark.read.option("header", "true").csv("hdfs://localhost:9000/user/ketkimaddiwar/airline_data/")
df.show(5)
# Now run count again
df.count()


25/05/03 13:47:01 WARN Utils: Your hostname, Shubhams-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.251.62.189 instead (on interface en0)
25/05/03 13:47:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/03 13:47:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/03 13:47:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/05/03 13:47:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, IATA_CODE_Reporting_Airline, Tail_Number, Flight_Number_Reporting_Airline, OriginAirportID, O

+----+-------+-----+----------+---------+----------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----------------+----

7079061

In [4]:
df.columns


['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'DOT_ID_Reporting_Airline',
 'IATA_CODE_Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityD

In [5]:
weather_delay_df = df.filter(df.WeatherDelay > 0)

In [6]:
weather_delay_df.count()


88905

In [46]:
weather_delay_df.select("DepDelay","DepDelayMinutes","ArrDelay","ArrDelayMinutes", "WeatherDelay","FlightDate").show(20, truncate=False)


+--------+---------------+--------+---------------+------------+----------+
|DepDelay|DepDelayMinutes|ArrDelay|ArrDelayMinutes|WeatherDelay|FlightDate|
+--------+---------------+--------+---------------+------------+----------+
|33.00   |33.00          |27.00   |27.00          |7.00        |2024-06-21|
|22.00   |22.00          |29.00   |29.00          |14.00       |2024-06-29|
|48.00   |48.00          |38.00   |38.00          |7.00        |2024-06-25|
|146.00  |146.00         |138.00  |138.00         |138.00      |2024-06-20|
|35.00   |35.00          |41.00   |41.00          |13.00       |2024-06-07|
|84.00   |84.00          |76.00   |76.00          |76.00       |2024-06-22|
|60.00   |60.00          |38.00   |38.00          |6.00        |2024-06-17|
|173.00  |173.00         |142.00  |142.00         |142.00      |2024-06-30|
|123.00  |123.00         |125.00  |125.00         |123.00      |2024-06-30|
|105.00  |105.00         |90.00   |90.00          |90.00       |2024-06-14|
|110.00  |11

In [7]:
total_flights = df.count()
weather_delayed = weather_delay_df.count()
percentage = (weather_delayed / total_flights) * 100
print(f"Weather related delays: {weather_delayed} ({percentage:.2f}% of total)")


Weather related delays: 88905 (1.26% of total)


In [10]:
from pyspark.sql.functions import col

# Filter only the flights where WeatherDelay > 0
weather_df = df.filter(col("WeatherDelay") > 0)

# 1. Only Departure Delay due to Weather
only_dep_weather = weather_df.filter(
    (col("DepDelay") >= col("WeatherDelay")) &
    (col("ArrDelay") < col("WeatherDelay"))
)

# 2. Only Arrival Delay due to Weather
only_arr_weather = weather_df.filter(
    (col("DepDelay") < col("WeatherDelay")) &
    (col("ArrDelay") >= col("WeatherDelay"))
)

# 3. Both Departure and Arrival Delays due to Weather
both_weather = weather_df.filter(
    (col("DepDelay") >= col("WeatherDelay")) &
    (col("ArrDelay") >= col("WeatherDelay"))
)

# 4. Unattributed Weather Delay (not clearly visible in dep or arr delay)
unattributed_weather = weather_df.filter(
    (col("DepDelay") < col("WeatherDelay")) &
    (col("ArrDelay") < col("WeatherDelay"))
)

# Count each group
print("Total WeatherDelay flights:", weather_df.count())
print("Only Departure Weather Delays:", only_dep_weather.count())
print("Only Arrival Weather Delays:", only_arr_weather.count())
print("Both Departure & Arrival Weather Delays:", both_weather.count())
print("Unattributed Weather Delays:", unattributed_weather.count())


Total WeatherDelay flights: 88905


Only Departure Weather Delays: 7870


Only Arrival Weather Delays: 2691


Both Departure & Arrival Weather Delays: 65687


Unattributed Weather Delays: 12657


In [11]:
###########################

In [12]:
from pyspark.sql.functions import col

# Total flights delayed due to weather
weather_delayed_flights = df.filter(col("WeatherDelay") > 0)
weather_delayed_count = weather_delayed_flights.count()

print(f"Total flights delayed due to weather: {weather_delayed_count}")


Total flights delayed due to weather: 88905


In [13]:
from pyspark.sql.functions import avg, min, max

# Summary statistics for weather delays
weather_delay_stats = weather_delayed_flights.select("WeatherDelay").agg(
    avg("WeatherDelay").alias("avg_weather_delay"),
    min("WeatherDelay").alias("min_weather_delay"),
    max("WeatherDelay").alias("max_weather_delay")
)

weather_delay_stats.show()


+-----------------+-----------------+-----------------+
|avg_weather_delay|min_weather_delay|max_weather_delay|
+-----------------+-----------------+-----------------+
| 69.6909397671672|             1.00|           997.00|
+-----------------+-----------------+-----------------+



In [14]:
#averge weather delays by month
from pyspark.sql.functions import month

# Add month column if not already present (use FlightDate if available)
monthly_weather_delays = weather_delayed_flights.groupBy("Month").count().orderBy("Month")

monthly_weather_delays.show(12)


+-----+-----+
|Month|count|
+-----+-----+
|    1|12282|
|   10| 1830|
|   11| 3890|
|   12| 7939|
|    2| 3692|
|    3| 5765|
|    4| 5921|
|    5|12205|
|    6| 9757|
|    7|11861|
|    8| 9229|
|    9| 4534|
+-----+-----+



In [15]:
#Carrier-wise Weather Delays
carrier_weather_delays = weather_delayed_flights.groupBy("Reporting_Airline").count().orderBy("count", ascending=False)
carrier_weather_delays.show(truncate=False)


+-----------------+-----+
|Reporting_Airline|count|
+-----------------+-----+
|AA               |20217|
|OO               |17677|
|MQ               |8305 |
|DL               |7930 |
|UA               |6991 |
|WN               |6791 |
|OH               |5254 |
|NK               |2792 |
|G4               |2653 |
|YX               |2586 |
|AS               |2068 |
|9E               |1890 |
|B6               |1577 |
|F9               |1426 |
|HA               |748  |
+-----------------+-----+



In [16]:
# weather-related delays by day of the week
# Group weather delay records by DayOfWeek and count them
weather_by_day = df.filter(df.WeatherDelay > 0) \
    .groupBy("DayOfWeek") \
    .count() \
    .orderBy("DayOfWeek")

weather_by_day.show()


+---------+-----+
|DayOfWeek|count|
+---------+-----+
|        1|13860|
|        2|13044|
|        3|10711|
|        4|13279|
|        5|13211|
|        6|11217|
|        7|13583|
+---------+-----+



In [17]:
#Average weather delay duration by origin airport

from pyspark.sql.functions import col

# Create a new DataFrame with WeatherDelay casted to double
df_casted = df.withColumn("WeatherDelay", col("WeatherDelay").cast("double"))

# Now perform the aggregation
avg_weather_delay_by_origin = df_casted.filter(col("WeatherDelay") > 0) \
    .groupBy("Origin") \
    .avg("WeatherDelay") \
    .withColumnRenamed("avg(WeatherDelay)", "AvgWeatherDelay") \
    .orderBy("AvgWeatherDelay", ascending=False)

avg_weather_delay_by_origin.show(20, truncate=False)


+------+------------------+
|Origin|AvgWeatherDelay   |
+------+------------------+
|SPN   |505.5             |
|HYS   |473.5             |
|STC   |458.0             |
|EAR   |389.0             |
|DEC   |325.1666666666667 |
|LBF   |307.5             |
|MCW   |266.5             |
|PVU   |241.1304347826087 |
|JST   |237.75            |
|WYS   |236.1818181818182 |
|OTH   |230.625           |
|DIK   |222.22222222222223|
|SUX   |211.64285714285714|
|PBG   |208.8125          |
|PIB   |208.5             |
|JMS   |208.125           |
|BFF   |201.25            |
|PSM   |199.83333333333334|
|ACV   |177.06521739130434|
|VCT   |175.33333333333334|
+------+------------------+
only showing top 20 rows



In [18]:

#Top 20 Arrival Airports with the Most Weather Delays
arrival_weather_delay_airports = df_casted.filter(col("WeatherDelay") > 0) \
    .groupBy("Dest") \
    .count() \
    .withColumnRenamed("count", "WeatherDelayCount") \
    .orderBy("WeatherDelayCount", ascending=False)

arrival_weather_delay_airports.show(20, truncate=False)


+----+-----------------+
|Dest|WeatherDelayCount|
+----+-----------------+
|DFW |8271             |
|CLT |3824             |
|ATL |3696             |
|ORD |3532             |
|DEN |2645             |
|MSP |2205             |
|DTW |2190             |
|LAX |2082             |
|LGA |2045             |
|SEA |1956             |
|DCA |1711             |
|PHX |1571             |
|LAS |1545             |
|SFO |1495             |
|MIA |1492             |
|MCO |1436             |
|PHL |1409             |
|BOS |1397             |
|SLC |1363             |
|EWR |1346             |
+----+-----------------+
only showing top 20 rows



In [19]:
from pyspark.sql.functions import col

arrival_weather_df = df_casted.filter(
    (col("WeatherDelay") > 0) &
    (col("DepDelay") < col("WeatherDelay")) &
    (col("ArrDelay") >= col("WeatherDelay"))
)

arrival_airport_weather_delay = arrival_weather_df.groupBy("Dest") \
    .count() \
    .withColumnRenamed("count", "ArrivalWeatherDelayCount") \
    .orderBy("ArrivalWeatherDelayCount", ascending=False)

arrival_airport_weather_delay.show(10, truncate=False)


+----+------------------------+
|Dest|ArrivalWeatherDelayCount|
+----+------------------------+
|HNL |166                     |
|PIE |74                      |
|SFB |28                      |
|LIH |26                      |
|OGG |15                      |
|PGD |12                      |
|KOA |12                      |
|ITO |10                      |
|BNA |7                       |
|VPS |6                       |
+----+------------------------+
only showing top 10 rows



In [20]:
from pyspark.sql.functions import col

# Filter only the rows with WeatherDelay > 0
weather_df = df.filter(col("WeatherDelay") > 0)

# Category 1: Only Arrival-caused
arrival_only = weather_df.filter(
    (col("DepDelay") < col("WeatherDelay")) & 
    (col("ArrDelay") >= col("WeatherDelay"))
)
arrival_count = arrival_only.count()
print(f"Arrival City Caused Delays: {arrival_count}")

# Category 2: Only Departure-caused
departure_only = weather_df.filter(
    (col("DepDelay") >= col("WeatherDelay")) & 
    (col("ArrDelay") < col("WeatherDelay"))
)
departure_count = departure_only.count()
print(f"Departure City Caused Delays: {departure_count}")

# Category 3: Both Cities Likely Involved
both_cities = weather_df.filter(
    (col("DepDelay") >= col("WeatherDelay")) & 
    (col("ArrDelay") >= col("WeatherDelay"))
)
both_count = both_cities.count()
print(f"Both Cities Likely Involved: {both_count}")

# Total rows with WeatherDelay > 0
total_weather_delays = weather_df.count()
print(f"Total Weather Delay Rows: {total_weather_delays}")

# Category 4: Unattributed (those that didn’t fall in above 3)
known_total = arrival_count + departure_count + both_count
unattributed_count = total_weather_delays - known_total
print(f"Unattributed Weather Delay Rows: {unattributed_count}")


Arrival City Caused Delays: 2691


Departure City Caused Delays: 7870


Both Cities Likely Involved: 65687


Total Weather Delay Rows: 88905
Unattributed Weather Delay Rows: 12657


In [21]:
from pyspark.sql.functions import col, count

# Filter rows where WeatherDelay > 0
weather_delay_df = df.filter(col("WeatherDelay") > 0)

# Group by Origin and Dest and count
airport_pairs_delay_count = weather_delay_df.groupBy("Origin", "Dest") \
    .agg(count("*").alias("WeatherDelayCount")) \
    .orderBy(col("WeatherDelayCount").desc())

# Show the top 20 origin-destination pairs
airport_pairs_delay_count.show(20, truncate=False)


+------+----+-----------------+
|Origin|Dest|WeatherDelayCount|
+------+----+-----------------+
|LIT   |DFW |271              |
|MCO   |DFW |271              |
|MCO   |CLT |211              |
|XNA   |DFW |209              |
|MCO   |PHL |193              |
|MAF   |DFW |191              |
|DFW   |LAX |187              |
|MCO   |ATL |173              |
|MCO   |MIA |172              |
|DFW   |AUS |167              |
|DFW   |PHX |165              |
|ORD   |LGA |163              |
|LFT   |DFW |163              |
|DFW   |LAS |160              |
|DFW   |SAT |155              |
|DFW   |ATL |155              |
|DFW   |LGA |153              |
|IAH   |DFW |152              |
|MIA   |DFW |152              |
|ORD   |EWR |151              |
+------+----+-----------------+
only showing top 20 rows



In [22]:
# Count of unique origin-destination pairs with weather delay
unique_weather_routes_count = weather_delay_df.select("Origin", "Dest").distinct().count()

print("Total unique origin-destination pairs with weather delay:", unique_weather_routes_count)


Total unique origin-destination pairs with weather delay: 5541


In [40]:
from pyspark.sql.functions import to_date

# Convert FlightDate to proper date format (yyyy-MM-dd)
df = df.withColumn("FlightDate", to_date("FlightDate", "yyyy-MM-dd"))

# Confirm schema
df.select("FlightDate").show(5)

+----------+
|FlightDate|
+----------+
|2024-06-07|
|2024-06-08|
|2024-06-09|
|2024-06-10|
|2024-06-11|
+----------+
only showing top 5 rows



In [26]:
from pyspark.sql.functions import col

# Step 1: Extract Origin airport and date for weather-affected flights
weather_keys_df = df.filter(col("WeatherDelay") > 0) \
    .select("Origin", "FlightDate") \
    .distinct()

# Show a sample
weather_keys_df.count()
weather_keys_df.show(5)


+------+----------+
|Origin|FlightDate|
+------+----------+
|   TLH|2024-06-04|
|   BDL|2024-06-25|
|   TPA|2024-06-24|
|   GSO|2024-06-14|
|   MSY|2024-06-06|
+------+----------+
only showing top 5 rows



In [29]:
# Combine Origin and Dest columns and get distinct airport codes
unique_airports_df = df.select("Origin").union(df.select("Dest")).distinct()

# Convert to list of codes
airport_codes = [row["Origin"] for row in unique_airports_df.collect()]
print("Total unique airport codes found:", len(airport_codes))
print(airport_codes[:10])  # Print first 10 as a sample


Total unique airport codes found: 348
['BGM', 'DLG', 'PSE', 'MSY', 'PPG', 'GEG', 'SNA', 'BUR', 'GRB', 'GTF']


In [36]:
import pandas as pd

columns = ["AirportID", "Name", "City", "Country", "IATA", "ICAO", "Latitude", "Longitude",
           "Altitude", "Timezone", "DST", "TzDatabaseTimeZone", "Type", "Source"]

airports_df = pd.read_csv(
    "/Users/ketkimaddiwar/Documents/SJSU_Study/Data_228/Project/airports.dat.txt",
    header=None,
    names=columns
)

# Filter rows with valid IATA codes (3-letter airport codes)
airports_df = airports_df[airports_df["IATA"].notnull() & (airports_df["IATA"].str.len() == 3)]

airports_df.head()


,AirportID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TzDatabaseTimeZone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [33]:
# Step 3: Create mapping of IATA → (latitude, longitude)
airport_coordinates = {
    row["IATA"]: (row["Latitude"], row["Longitude"])
    for _, row in airports_df.iterrows()
}

# Example: Print coordinates for JFK and SFO
print("JFK:", airport_coordinates.get("JFK"))
print("SFO:", airport_coordinates.get("SFO"))


JFK: (40.63980103, -73.77890015)
SFO: (37.61899948120117, -122.375)
